In [3]:
# Import the requests library.
import requests
import json
import numpy as np
import pandas as pd
from citipy import citipy as cp
import matplotlib.pyplot as plt

# Import the API key.
from config import weather_api_key

# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime

In [4]:
latitudes = np.random.uniform(-90.000, 90.000, 2000)
longitudes = np.random.uniform(-180.000,180.000,2000)

lat_lngs = zip(latitudes, longitudes)
coords = list(lat_lngs)

In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coords:
    city = cp.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

778

In [6]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [ ]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
  #      time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
   # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_descr = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date, 
                          "Current Description" : city_weather_descr})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        print(city_url)
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


In [8]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]

city_data_df = city_data_df[new_column_order]

In [9]:
city_data_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Atuona,PF,2022-02-04 01:58:35,-9.8000,-139.0333,78.91,72,4,16.35,clear sky
1,Kapaa,US,2022-02-04 01:57:23,22.0752,-159.3190,78.78,66,0,1.99,clear sky
2,Soyo,AO,2022-02-04 01:56:43,-6.1349,12.3689,77.20,75,93,3.04,overcast clouds
3,Sylvania,US,2022-02-04 01:59:20,41.7189,-83.7130,21.04,89,100,7.00,overcast clouds
4,General Pico,AR,2022-02-04 01:59:20,-35.6566,-63.7568,73.81,61,53,7.99,light rain
...,...,...,...,...,...,...,...,...,...,...
713,Irece,BR,2022-02-04 02:03:52,-11.3042,-41.8558,71.08,66,25,8.23,scattered clouds
714,Barcelos,PT,2022-02-04 02:03:52,41.5388,-8.6151,51.94,77,100,2.06,overcast clouds
715,Novopokrovka,UA,2022-02-04 02:03:53,49.8425,36.5477,30.58,89,100,6.85,overcast clouds
716,Narrabri,AU,2022-02-04 02:03:53,-30.3167,149.7833,78.53,44,20,17.27,few clouds


In [10]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")